# CAPSTONE
Kehinde Ajayi

Initial modeling

In [1]:
## check if notebook is running on Colab--if so, run appropriate installs and mount Google Drive
# https://stackoverflow.com/questions/53581278/test-if-notebook-is-running-on-google-colab

if 'google.colab' in str(get_ipython()):
  # install RDKit and DeepChem
  !pip install rdkit-pypi scikit-learn --pre deepchem tensorflow-addons
  # mount Google Drive
  from google.colab import drive
  drive.mount('/content/drive')

In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import datetime
import sys

from tqdm.auto import tqdm

from rdkit.Chem import PandasTools

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score, f1_score, ConfusionMatrixDisplay, accuracy_score, make_scorer, log_loss, classification_report, roc_auc_score, recall_score, RocCurveDisplay, matthews_corrcoef
from sklearn.pipeline import Pipeline

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import torch.nn as nn
import torch.nn.functional as F

import tensorboard


import deepchem as dc


from hyperopt import hp, fmin, tpe, Trials

from lime import lime_tabular


In [3]:
tf.config.list_physical_devices('GPU')

2022-05-14 00:07:29.475826: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-14 00:07:29.475930: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


[]

## Functions and settings

In [10]:
# workaround for bug where molecular structure does not print with dataframe
# https://github.com/rdkit/rdkit/issues/2673
def show(df):
    return HTML(df.to_html(notebook=True))


In [11]:
tqdm.pandas()

In [12]:
# check if notebook is running on Colab and set path for data file 

def get_home_path():
  if 'google.colab' in str(get_ipython()):
    path = '/content/drive/MyDrive/GA/Capstone/ka_capstone'
  else:
    path = '..'
  
  return path


In [13]:
# view all columns in dataframe
pd.set_option('display.max.columns', None)

## Read data

In [14]:
# # read hERG data into dataframe 
# herg_df = PandasTools.LoadSDF(f'{get_home_path()}/data/interim/herg-data-with-structures.sdf', molColName='Mol', smilesName='SMILES', includeFingerprints=True, idName='Pubchem_SID')


In [15]:
# read hERG data without structure fingerprint into separate dataframe 
df = pd.read_csv(f'{get_home_path()}/data/interim/herg_data.csv')



In [16]:
df.head()

,Pubchem_SID,SMILES,hERG_at_1uM,hERG_at_10uM,herg_inhibitor,mw,logP,num_stereocenters,num_heteroatoms,total_num_rings,aromatic_rings,num_rotatable_bonds,PSA,formal_charge,H_bond_donors,H_bond_acceptors,SMILES_length,ro5_violations
0,22416348,Cc1occc1C(=O)NCc1ccco1,20.17528,30.99165,0,205.213,2.11102,0,4,2,2,4,55.38,0,1,3,22,0
1,26665387,COc1ccc(/C=C2\SC(=S)N(N3CCOCC3)C2=O)c(OC)c1,10.22630,13.05888,0,366.464,2.15220,0,8,3,1,8,51.24,0,0,7,43,0
2,862531,C[C@H](NC(=O)Nc1cccc(C(F)(F)F)c1)C(=O)O,2.04420,0.06288,0,276.214,2.30000,1,8,1,1,8,78.43,0,3,2,39,0
3,26732361,COc1cc(OC)c2ccc(=O)oc2c1C(CC(=O)N1CCOCC1)c1ccc...,21.80250,17.87858,0,466.534,3.25700,1,8,4,3,8,81.45,0,0,7,56,0
4,49735227,COc1cccc(NC(=O)C2C3C=CC4(O3)C2C(=O)N(CCCN2CCCC...,8.33980,19.03128,0,550.700,3.10920,5,9,6,1,9,100.21,0,2,6,68,1


In [10]:
# add image of molecule to dataframe
PandasTools.AddMoleculeColumnToFrame(df, 'SMILES', 'Mol')

In [8]:
df.info()

NameError: name 'df' is not defined

## Initial model without molecular fingerprint

### Create dataset for model


In [17]:
## load dataset if it exists and create it if it doesn't
try:
    # load existing dataset from disk
    dataset = dc.data.DiskDataset('../data/processed/fingerprints')
except ValueError:
    # create dataset
    dataset_file = f'{get_home_path()}/data/interim/herg_data.csv'
    tasks = ['herg_inhibitor']
    featurizer = dc.feat.CircularFingerprint(size=1024)
    loader = dc.data.CSVLoader(tasks=tasks,
                               feature_field='SMILES',
                               featurizer=featurizer)
    dataset = loader.create_dataset(dataset_file, f'{get_home_path()}/data/processed/fingerprints', shard_size=8192)

In [18]:
dataset

<DiskDataset X.shape: (306865, 1024), y.shape: (306865, 1), w.shape: (306865, 1), task_names: ['herg_inhibitor']>

In [14]:
# # split dataset into training, validation, and testing sets
# splitter = dc.splits.RandomStratifiedSplitter()
# train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset)

In [15]:
# # instantiate model
# model_1 = dc.models.MultitaskClassifier(n_tasks=1, n_features=1024, layer_sizes=[512, 128, 32])

In [16]:
# define metrics for evaluating models
metrics = [dc.metrics.Metric(dc.metrics.matthews_corrcoef, np.mean, mode='classification'), dc.metrics.Metric(dc.metrics.accuracy_score, np.mean, mode='classification')]

In [24]:
# split dataset into training, validation, and testing sets
splitter = dc.splits.RandomStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset)

# instantiate model
model_1 = dc.models.MultitaskClassifier(n_tasks=1, n_features=1024, layer_sizes=[512, 128, 32])

# fit model
model_1.fit(train_dataset, nb_epoch=10)

print('training set score:', model_1.evaluate(train_dataset, metrics))
print('test set score:', model_1.evaluate(test_dataset, metrics))

training set score: {'mean-matthews_corrcoef': 0.7128495082375357, 'mean-accuracy_score': 0.9784229221318821}
test set score: {'mean-matthews_corrcoef': 0.43353692088295115, 'mean-accuracy_score': 0.9621663896764102}


In [25]:
model_1.save('../models//fingerprint_features/fp_randstratsplit.h5')

TypeError: save() takes 1 positional argument but 2 were given

In [82]:
model_1.history

AttributeError: 'MultitaskClassifier' object has no attribute 'history'

In [27]:
# split dataset into training, validation, and testing sets using scaffold
splitter = dc.splits.ScaffoldSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset)

# instantiate model
model_2 = dc.models.MultitaskClassifier(n_tasks=1, n_features=1024, layer_sizes=[512, 128, 32])

# fit 
model_2.fit(train_dataset, nb_epoch=10)

print('training set score:', model_2.evaluate(train_dataset, metrics))
print('test set score:', model_2.evaluate(test_dataset, metrics))

training set score: {'mean-matthews_corrcoef': 0.659313041507731, 'mean-accuracy_score': 0.9761825232594138}
test set score: {'mean-matthews_corrcoef': 0.2663299162367317, 'mean-accuracy_score': 0.9525857855117802}


Stratified splitting seems to work better than scaffold splitting (MCC coefficient of 0.43 vs 0.27 for test sets).  Let's try Robust Multitask Classifier with stratified splitting

In [30]:
# split dataset into training, validation, and testing sets
splitter = dc.splits.RandomStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset)

# instantiate model
model_3 = dc.models.RobustMultitaskClassifier(n_tasks=1, n_features=1024, layer_sizes=[512, 128, 32])

# fit model
model_3.fit(train_dataset, nb_epoch=10)

print('training set score:', model_3.evaluate(train_dataset, metrics))
print('test set score:', model_3.evaluate(test_dataset, metrics))

2022-05-12 18:06:41.194757: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-12 18:06:41.197604: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-12 18:06:41.198289: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-12 18:06:41.198816: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retur

training set score: {'mean-matthews_corrcoef': 0.5399108330113764, 'mean-accuracy_score': 0.9686710768579017}
test set score: {'mean-matthews_corrcoef': 0.38205464887558316, 'mean-accuracy_score': 0.9615146478965034}


### cross-validated training

In [39]:
training_score_list = []
validation_score_list = []
cv_folds = 5
splitter = dc.splits.RandomStratifiedSplitter()
for i in range(0, cv_folds):
model = dc.models.MultitaskClassifier(n_tasks=1, n_features=1024, layer_sizes=[512, 256, 32])
    train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset)
    model.fit(train_dataset)
    train_scores = model.evaluate(train_dataset,
                                  metrics)
    training_score_list.append((train_scores["mean-matthews_corrcoef"], train_scores["mean-accuracy_score"]))
    validation_scores = model.evaluate(valid_dataset,
                                       metrics)
    validation_score_list.append((validation_scores["mean-matthews_corrcoef"], validation_scores["mean-accuracy_score"]))
    print(training_score_list)
    print(validation_score_list)



[(0.6373733736733987, 0.9738199208120835)]
[(0.4413714843764347, 0.9630450368246106)]
[(0.6373733736733987, 0.9738199208120835), (0.631051317660848, 0.9734981180649471)]
[(0.4413714843764347, 0.9630450368246106), (0.4215217180840881, 0.9620999804471094)]
[(0.6373733736733987, 0.9738199208120835), (0.631051317660848, 0.9734981180649471), (0.6817837929188357, 0.9762639923093217)]
[(0.4413714843764347, 0.9630450368246106), (0.4215217180840881, 0.9620999804471094), (0.45889282834578615, 0.9626539790132308)]
[(0.6373733736733987, 0.9738199208120835), (0.631051317660848, 0.9734981180649471), (0.6817837929188357, 0.9762639923093217), (0.6366732291842369, 0.973836214622065)]
[(0.4413714843764347, 0.9630450368246106), (0.4215217180840881, 0.9620999804471094), (0.45889282834578615, 0.9626539790132308), (0.4039214535348725, 0.9616111581828847)]
[(0.6373733736733987, 0.9738199208120835), (0.631051317660848, 0.9734981180649471), (0.6817837929188357, 0.9762639923093217), (0.6366732291842369, 0.97383

- [(0.6373733736733987, 0.9738199208120835)]  
  [(0.4413714843764347, 0.9630450368246106)]  
- [(0.6373733736733987, 0.9738199208120835), (0.631051317660848, 0.9734981180649471)]  
  [(0.4413714843764347, 0.9630450368246106), (0.4215217180840881, 0.9620999804471094)]  
- [(0.6373733736733987, 0.9738199208120835), (0.631051317660848, 0.9734981180649471), (0.6817837929188357, 0.9762639923093217)]  
  [(0.4413714843764347, 0.9630450368246106), (0.4215217180840881, 0.9620999804471094), (0.45889282834578615, 0.9626539790132308)]  
- [(0.6373733736733987, 0.9738199208120835), (0.631051317660848, 0.9734981180649471), (0.6817837929188357, 0.9762639923093217), (0.6366732291842369, 0.973836214622065)]  
  [(0.4413714843764347, 0.9630450368246106), (0.4215217180840881, 0.9620999804471094), (0.45889282834578615, 0.9626539790132308), (0.4039214535348725, 0.9616111581828847)]  
- [(0.6373733736733987, 0.9738199208120835), (0.631051317660848, 0.9734981180649471), (0.6817837929188357, 0.9762639923093217), (0.6366732291842369, 0.973836214622065), (0.6724235643663177, 0.9759340426571945)]  
  [(0.4413714843764347, 0.9630450368246106), (0.4215217180840881, 0.9620999804471094), (0.45889282834578615, 0.9626539790132308), (0.4039214535348725, 0.9616111581828847), (0.4203265443595162, 0.9620348041452128)]  






-- Mean training MCC: 0.6518610555607275
   Mean training accuracy: 0.9746704576931224
**************************************************
-- Mean validation MCC: 0.42920680574013953
   Mean validation accuracy: 0.9622889917226096

In [77]:
print(f'-- Mean training MCC: {np.mean([x[0] for x in training_score_list])}\n   Mean training accuracy: {np.mean([x[1] for x in training_score_list])}')
print('*' * 50)
print(f'-- Mean validation MCC: {np.mean([x[0] for x in validation_score_list])}\n   Mean validation accuracy: {np.mean([x[1] for x in validation_score_list])}')                              

-- Mean training MCC: 0.6518610555607275
   Mean training accuracy: 0.9746704576931224
**************************************************
-- Mean validation MCC: 0.42920680574013953
   Mean validation accuracy: 0.9622889917226096


In [60]:
print(f'Mean training MCC: {np.mean([x[0] for x in training_score_list])}  Mean training accuracy: ')

Mean training MCC: 0.6518610555607275  Mean training accuracy: 


In [62]:
f'{np.mean([x[1] for x intraining_score_listg_score_list])}'

SyntaxError: f-string: invalid syntax (1314863352.py, line 1)

In [80]:
training_score_list = []
validation_score_list = []
cv_folds = 5
splitter = dc.splits.RandomStratifiedSplitter()
for i in range(0, cv_folds):
    model = dc.models.MultitaskClassifier(n_tasks=1, n_features=1024, layer_sizes=[1024, 256, 32])
    train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset)
    model.fit(train_dataset)
    train_scores = model.evaluate(train_dataset,
                                  metrics)
    training_score_list.append((train_scores["mean-matthews_corrcoef"], train_scores["mean-accuracy_score"]))
    validation_scores = model.evaluate(valid_dataset,
                                       metrics)
    validation_score_list.append((validation_scores["mean-matthews_corrcoef"], validation_scores["mean-accuracy_score"]))
    print(training_score_list)
    print(validation_score_list)

[(0.7625968428070329, 0.9817509328206214)]
[(0.4679055564607532, 0.9638923287492668)]
[(0.7625968428070329, 0.9817509328206214), (0.7778830329667884, 0.9827367083245075)]
[(0.4679055564607532, 0.9638923287492668), (0.45792036598163266, 0.9626539790132308)]
[(0.7625968428070329, 0.9817509328206214), (0.7778830329667884, 0.9827367083245075), (0.7624928822041234, 0.9817509328206214)]
[(0.4679055564607532, 0.9638923287492668), (0.45792036598163266, 0.9626539790132308), (0.43598474511669716, 0.9622303330509027)]
[(0.7625968428070329, 0.9817509328206214), (0.7778830329667884, 0.9827367083245075), (0.7624928822041234, 0.9817509328206214), (0.7952210997500605, 0.9838528343082463)]
[(0.4679055564607532, 0.9638923287492668), (0.45792036598163266, 0.9626539790132308), (0.43598474511669716, 0.9622303330509027), (0.48421003565520637, 0.9630776249755589)]
[(0.7625968428070329, 0.9817509328206214), (0.7778830329667884, 0.9827367083245075), (0.7624928822041234, 0.9817509328206214), (0.7952210997500605

In [81]:
print(f'-- Mean training MCC: {np.mean([x[0] for x in training_score_list])}\n   Mean training accuracy: {np.mean([x[1] for x in training_score_list])}')
print('*' * 50)
print(f'-- Mean validation MCC: {np.mean([x[0] for x in validation_score_list])}\n   Mean validation accuracy: {np.mean([x[1] for x in validation_score_list])}')                              

-- Mean training MCC: 0.7752154667361397
   Mean training accuracy: 0.9825753996056898
**************************************************
-- Mean validation MCC: 0.4591611111756123
   Mean validation accuracy: 0.9628951313302483


In [80]:
training_score_list = []
validation_score_list = []
cv_folds = 5
splitter = dc.splits.RandomStratifiedSplitter()
for i in range(0, cv_folds):
    model = dc.models.MultitaskClassifier(n_tasks=1, n_features=1024, layer_sizes=[1024, 256, 32])
    train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset)
    model.fit(train_dataset)
    train_scores = model.evaluate(train_dataset,
                                  metrics)
    training_score_list.append((train_scores["mean-matthews_corrcoef"], train_scores["mean-accuracy_score"]))
    validation_scores = model.evaluate(valid_dataset,
                                       metrics)
    validation_score_list.append((validation_scores["mean-matthews_corrcoef"], validation_scores["mean-accuracy_score"]))
    print(training_score_list)
    print(validation_score_list)

[(0.7625968428070329, 0.9817509328206214)]
[(0.4679055564607532, 0.9638923287492668)]
[(0.7625968428070329, 0.9817509328206214), (0.7778830329667884, 0.9827367083245075)]
[(0.4679055564607532, 0.9638923287492668), (0.45792036598163266, 0.9626539790132308)]
[(0.7625968428070329, 0.9817509328206214), (0.7778830329667884, 0.9827367083245075), (0.7624928822041234, 0.9817509328206214)]
[(0.4679055564607532, 0.9638923287492668), (0.45792036598163266, 0.9626539790132308), (0.43598474511669716, 0.9622303330509027)]
[(0.7625968428070329, 0.9817509328206214), (0.7778830329667884, 0.9827367083245075), (0.7624928822041234, 0.9817509328206214), (0.7952210997500605, 0.9838528343082463)]
[(0.4679055564607532, 0.9638923287492668), (0.45792036598163266, 0.9626539790132308), (0.43598474511669716, 0.9622303330509027), (0.48421003565520637, 0.9630776249755589)]
[(0.7625968428070329, 0.9817509328206214), (0.7778830329667884, 0.9827367083245075), (0.7624928822041234, 0.9817509328206214), (0.7952210997500605

In [81]:
print(f'-- Mean training MCC: {np.mean([x[0] for x in training_score_list])}\n   Mean training accuracy: {np.mean([x[1] for x in training_score_list])}')
print('*' * 50)
print(f'-- Mean validation MCC: {np.mean([x[0] for x in validation_score_list])}\n   Mean validation accuracy: {np.mean([x[1] for x in validation_score_list])}')                              

-- Mean training MCC: 0.7752154667361397
   Mean training accuracy: 0.9825753996056898
**************************************************
-- Mean validation MCC: 0.4591611111756123
   Mean validation accuracy: 0.9628951313302483


## Hyperparameter tuning

In [19]:
search_space = {
    'layer_sizes': hp.choice('layer_sizes',[[1024], [2048], [4096], [2048,512]]),
    'dropouts': hp.uniform('dropout',low=0.2, high=0.5),
    'learning_rate': hp.uniform('learning_rate',high=0.001, low=0.0001)
}

Best: {'dropout': 0.40467148362940397, 'layer_sizes': 2, 'learning_rate': 0.0002061284950914087}
one layer with 2048 neurons was best

In [20]:
import tempfile
#tempfile is used to save the best checkpoint later in the program.

In [21]:
# split dataset into training, validation, and testing sets
splitter = dc.splits.RandomStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset)

In [25]:
metric = dc.metrics.Metric(dc.metrics.matthews_corrcoef)

def fm(args):
    save_dir = tempfile.mkdtemp()
    model = dc.models.MultitaskClassifier(n_tasks=len(dataset.y[1]),
                                          n_features=1024,
                                          layer_sizes=args['layer_sizes'],
                                          dropouts=args['dropouts'],
                                          learning_rate=args['learning_rate'])
    #validation callback that saves the best checkpoint, i.e the one with the maximum score.
    validation=dc.models.ValidationCallback(valid_dataset,
                                            1000, [metric],save_dir=save_dir,save_on_minimum=False)

    model.fit(train_dataset, nb_epoch=25,callbacks=validation)

    #restoring the best checkpoint and passing the negative of its validation score to be minimized.
    model.restore(model_dir=save_dir)
    valid_score = model.evaluate(valid_dataset, [metric])

    return -1*valid_score['matthews_corrcoef']

In [26]:
len(dataset.y[1])

1

In [27]:
trials=Trials()
best = fmin(fm,
            space= search_space,
            algo=tpe.suggest,
            max_evals=15,
            trials = trials)

  0%|                               | 0/15 [00:00<?, ?trial/s, best loss=?]Step 1000 validation: matthews_corrcoef=0.248473
Step 2000 validation: matthews_corrcoef=0.257911
Step 3000 validation: matthews_corrcoef=0.258408
Step 4000 validation: matthews_corrcoef=0.217017
Step 5000 validation: matthews_corrcoef=0.31774
Step 6000 validation: matthews_corrcoef=0.365491
Step 7000 validation: matthews_corrcoef=0.399578
Step 8000 validation: matthews_corrcoef=0.439553
Step 9000 validation: matthews_corrcoef=0.418594
Step 10000 validation: matthews_corrcoef=0.430914
Step 11000 validation: matthews_corrcoef=0.427849
Step 12000 validation: matthews_corrcoef=0.415083
Step 13000 validation: matthews_corrcoef=0.406829
Step 14000 validation: matthews_corrcoef=0.436234
Step 15000 validation: matthews_corrcoef=0.460369
Step 16000 validation: matthews_corrcoef=0.457303
Step 17000 validation: matthews_corrcoef=0.436279
Step 18000 validation: matthews_corrcoef=0.450485
Step 19000 validation: matthews_cor

In [28]:
print("Best: {}".format(best))

Best: {'dropout': 0.40467148362940397, 'layer_sizes': 2, 'learning_rate': 0.0002061284950914087}
